# DeepLabCut Ingestion/Inference

`Dev notes:` Currently, the path structure assumes you have one DLC project directory for all models, as specified within `adamacs.pipeline.get_dlc_root_data_dir`. The parallel function `get_dlc_processed_data_dir` can specify the output directory. 

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

[2023-07-26 16:39:20,229][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-07-26 16:39:20,276][INFO]: Connected tobiasr@172.26.128.53:3306


DataJoint connection (connected) tobiasr@172.26.128.53:3306

In [2]:
dj.config["custom"].get("dlc_root_data_dir")


['/datajoint-data/data/tobiasr/DeepLabCutModels/DB_DLC_tracking',
 '/datajoint-data/data/tobiasr/DeepLabCutModels/NK_DLC_tracking',
 '/']

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates items.

In [3]:
from adamacs.pipeline import subject, train,  scan, event, trial, behavior, session, model, equipment

## Ingesting videos and training parameters

### Automated

Refer the `user_data` folder in the `adamacs` directory contains CSVs for inserting values into DeepLabCut tables.

1. `config_params.csv` is used for training parameter sets in `train.TrainingParamSet`. The following items are required, but others will also be passed to DLC's `train_network` function when called 
2. `train_videosets.csv` and `model_videos.csv` pass values to `train.VideoSet` and `model.VideoRecording` respectively.
3. `adamacs.ingest.dlc.ingest_dlc_items` will load each of these CSVs

For more information, see [this notebook](https://github.com/CBroz1/workflow-deeplabcut/blob/main/notebooks/04-Automate_Optional.ipynb)

In [4]:
    # from adamacs.ingest.dlc import ingest_dlc_items
    # ingest_dlc_items()

In [5]:
model.VideoRecording().delete()
model.Model.delete()

[2023-07-26 16:39:20,754][INFO]: Deleting 0 rows from `tobiasr_model`.`video_recording`
[2023-07-26 16:39:20,759][INFO]: Deleting 1 rows from `tobiasr_model`.`__model_evaluation`
[2023-07-26 16:39:20,763][INFO]: Deleting 7 rows from `tobiasr_model`.`model__body_part`
[2023-07-26 16:39:20,766][INFO]: Deleting 1 rows from `tobiasr_model`.`model`


Nothing to delete.
Deletes cancelled


1

In [6]:
train.schema.list_tables()

['#training_param_set',
 'video_set',
 'video_set__file',
 'training_task',
 '__model_training']

In [7]:
model.schema.list_tables()

['#body_part',
 'model',
 'model__body_part',
 'video_recording',
 'video_recording__file',
 '__model_evaluation',
 '_recording_info',
 'pose_estimation_task',
 '__pose_estimation',
 '__pose_estimation__body_part_position']

### Manual

The same training parameters as above can be manually inserted as follows.

In [8]:
import yaml
from element_interface.utils import find_full_path
from adamacs.paths import get_dlc_root_data_dir
import pathlib
config_path = find_full_path(get_dlc_root_data_dir(), 
                             'from_top_tracking-DJ-2022-02-23/config.yaml')
config_path

PosixPath('/datajoint-data/data/tobiasr/DeepLabCutModels/DB_DLC_tracking/from_top_tracking-DJ-2022-02-23/config.yaml')

In [9]:
# model.VideoRecording.delete()

In [10]:
scansi = "scan9FJ842C3"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])

key = {'session_id': scan_key["session_id"],
       'recording_id': scan_key["scan_id"], 
       'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
}
model.VideoRecording.insert1(key, skip_duplicates=True)
# do not include an initial `/` in relative file paths   
key.update({'file_path': moviepath,
            'file_id': 0})
model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)

In [11]:
scansi = "scan9FB2LN5C"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])

key = {'session_id': scan_key["session_id"],
       'recording_id': scan_key["scan_id"], 
       'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
}
model.VideoRecording.insert1(key, skip_duplicates=True)
# do not include an initial `/` in relative file paths   
key.update({'file_path': moviepath,
            'file_id': 1})
model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)

In [12]:
scansi = "scan9FHELAYA"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*53.mp4"))[0])
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4-copy.mp4"))[0])

key = {'session_id': scan_key["session_id"],
       'recording_id': scan_key["scan_id"], 
       'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
}
model.VideoRecording.insert1(key, skip_duplicates=True)
# do not include an initial `/` in relative file paths   
key.update({'file_path': moviepath,
            'file_id': 2})
model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)

In [13]:
model.VideoRecording() * model.VideoRecording.File() * equipment.Device()

session_id,recording_id,file_id,camera,"file_path filepath of video, relative to root data directory",camera_description
sess9FB2LN5C,scan9FB2LN5C,1,mini2p1_top,/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_top_video_2022-03-18T16_55_33.mp4,"Basler a2A1920-160umBAS, Xx objective"
sess9FHELAYA,scan9FHELAYA,2,mini2p1_top,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4-copy.mp4,"Basler a2A1920-160umBAS, Xx objective"
sess9FJ842C3,scan9FJ842C3,0,mini2p1_top,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-07-24_scan9FJ842C3_sess9FJ842C3/scan9FJ842C3_top_video_2023-07-24T16_38_06.mp4,"Basler a2A1920-160umBAS, Xx objective"


## Model Training

The `TrainingTask` table queues up training. To launch training from a different machine, one needs to edit DLC's config files to reflect updated paths. For training, this includes `dlc-models/*/*/train/pose_cfg.yaml`

`CB DEV NOTE:` I'm missing the following videos used to originally train the model:
- top_video2022-02-17T15_56_10.mp4
- top_video2022-02-21T12_18_09.mp4

#### DeepLabcut Tables

The `VideoSet` table in the `train` schema retains records of files generated in the video labeling process (e.g., `h5`, `csv`, `png`). DeepLabCut will refer to the `mat` file located under the `training-datasets` directory.

We recommend storing all paths as relative to the root in your config.

In [14]:
# train.VideoSet.delete()

In [15]:
train.VideoSet.insert1({'video_set_id': 0}, skip_duplicates=True)
project_folder = 'from_top_tracking-DJ-2022-02-23/'
training_files = ['labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.h5',
                  'labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.csv',
                  'labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/img00674.png',
                  'videos/exp9FANLWRZ_top_video2022-02-21T12_18_09.mp4']
for idx, filename in enumerate(training_files):
    train.VideoSet.File.insert1({'video_set_id': 0,
                                 'file_id': idx,
                                 'file_path': (project_folder + filename)}, skip_duplicates=True)
train.VideoSet.File()

video_set_id,file_id,file_path
0,0,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.h5
0,1,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.csv
0,2,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/img00674.png
0,3,from_top_tracking-DJ-2022-02-23/videos/exp9FANLWRZ_top_video2022-02-21T12_18_09.mp4
1,0,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.h5
1,1,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.csv
1,2,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/img00162.png
1,3,Head_orientation-NK-2023-07-17/videos/scan9FHF1JT7_top_video_2023-04-06T09_31_19.mp4


In [16]:
train.VideoSet.insert1({'video_set_id': 1}, skip_duplicates=True)
project_folder = 'Head_orientation-NK-2023-07-17/'
training_files = ['labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.h5',
                  'labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.csv',
                  'labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/img00162.png',
                  'videos/scan9FHF1JT7_top_video_2023-04-06T09_31_19.mp4']
for idx, filename in enumerate(training_files):
    train.VideoSet.File.insert1({'video_set_id': 1,
                                 'file_id': idx,
                                 'file_path': (project_folder + filename)}, skip_duplicates=True)
train.VideoSet.File()

video_set_id,file_id,file_path
0,0,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.h5
0,1,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/CollectedData_DJ.csv
0,2,from_top_tracking-DJ-2022-02-23/labeled-data/exp9FANLWRZ_top_video2022-02-21T12_18_09/img00674.png
0,3,from_top_tracking-DJ-2022-02-23/videos/exp9FANLWRZ_top_video2022-02-21T12_18_09.mp4
1,0,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.h5
1,1,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/CollectedData_NK.csv
1,2,Head_orientation-NK-2023-07-17/labeled-data/scan9FHF1JT7_top_video_2023-04-06T09_31_19/img00162.png
1,3,Head_orientation-NK-2023-07-17/videos/scan9FHF1JT7_top_video_2023-04-06T09_31_19.mp4


In [17]:
# train.VideoSet().delete()

The `params` longblob should be a dictionary that captures all items for DeepLabCut's `train_network` function. At minimum, this is the contents of the project's config file, as well as `suffle` and `trainingsetindex`, which are not included in the config. 

In [18]:
from deeplabcut import train_network
help(train_network) # for more information on optional parameters

2023-07-26 16:39:30.319555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 16:39:30.446259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tobiasr/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-07-26 16:39:30.446279: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-26 16:39:30.476086: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already 

Loading DLC 2.3.4...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Help on function train_network in module deeplabcut.pose_estimation_tensorflow.training:

train_network(config, shuffle=1, trainingsetindex=0, max_snapshots_to_keep=5, displayiters=None, saveiters=None, maxiters=None, allow_growth=True, gputouse=None, autotune=False, keepdeconvweights=True, modelprefix='')
    Trains the network with the labels in the training dataset.
    
    Parameters
    ----------
    config : string
        Full path of the config.yaml file as a string.
    
    shuffle: int, optional, default=1
        Integer value specifying the shuffle index to select for training.
    
    trainingsetindex: int, optional, default=0
        Integer specifying which TrainingsetFraction to use.
        Note that TrainingFraction is a list in config.yaml.
    
    max_snapshots_to_keep: int or None
        Sets how many snapshots are kept, i.e. states of the trained n

Here, we give these items, load the config contents, and overwrite some defaults, including `maxiters`, to restrict our training iterations to 5.

In [19]:
train.TrainingParamSet()

paramset_idx,paramset_desc,param_set_hash hash identifying this parameterset,params dictionary of all applicable parameters
0,from_top_tracking-DJ-2022-02-23,d5736a15-23a7-5469-c521-6929b5f1d1b7,=BLOB=
1,Head_orientation-NK-2023-07-17,c6ec66ba-91f0-b916-fcbd-6dcf9f5adc23,=BLOB=


In [20]:
import yaml

paramset_idx = 0; paramset_desc='from_top_tracking-DJ-2022-02-23'

config_path = find_full_path(get_dlc_root_data_dir()[0], 
                             paramset_desc + '/config.yaml')

with open(config_path, 'rb') as y:
    config_params = yaml.safe_load(y)
training_params = {'shuffle': '1',
                   'trainingsetindex': '0',
                   'maxiters': '5',
                   'scorer_legacy': 'False',
                   'maxiters': '5', 
                   'multianimalproject':'False'}
config_params.update(training_params)
train.TrainingParamSet.insert_new_params(paramset_idx=paramset_idx,
                                         paramset_desc=paramset_desc,
                                         params=config_params)

In [21]:
paramset_idx = 1; paramset_desc='Head_orientation-NK-2023-07-17'

config_path = find_full_path(get_dlc_root_data_dir()[1], 
                             paramset_desc + '/config.yaml')

with open(config_path, 'rb') as y:
    config_params = yaml.safe_load(y)
training_params = {'shuffle': '1',
                   'trainingsetindex': '0',
                   'maxiters': '5',
                   'scorer_legacy': 'False',
                   'maxiters': '5', 
                   'multianimalproject':'False'}
config_params.update(training_params)
train.TrainingParamSet.insert_new_params(paramset_idx=paramset_idx,
                                         paramset_desc=paramset_desc,
                                         params=config_params)

In [22]:
train.TrainingParamSet()

paramset_idx,paramset_desc,param_set_hash hash identifying this parameterset,params dictionary of all applicable parameters
0,from_top_tracking-DJ-2022-02-23,d5736a15-23a7-5469-c521-6929b5f1d1b7,=BLOB=
1,Head_orientation-NK-2023-07-17,c6ec66ba-91f0-b916-fcbd-6dcf9f5adc23,=BLOB=


In [23]:
# train.TrainingTask.delete()

In [24]:
key={'video_set_id': 0, 
     'paramset_idx':0,
     'training_id':0, # uniquely defines training task
     'project_path':'from_top_tracking-DJ-2022-02-23/' # relative to dlc_root in dj.config
    }
train.TrainingTask.insert1(key, skip_duplicates=True)
train.TrainingTask()

video_set_id,paramset_idx,training_id,model_prefix,project_path DLC's project_path in config relative to root
0,0,0,,from_top_tracking-DJ-2022-02-23/
1,1,1,,Head_orientation-NK-2023-07-17


In [25]:
key={'video_set_id': 1, 
     'paramset_idx':1,
     'training_id':1, # uniquely defines training task
     'project_path':'Head_orientation-NK-2023-07-17' # relative to dlc_root in dj.config
    }
train.TrainingTask.insert1(key, skip_duplicates=True)
train.TrainingTask()

video_set_id,paramset_idx,training_id,model_prefix,project_path DLC's project_path in config relative to root
0,0,0,,from_top_tracking-DJ-2022-02-23/
1,1,1,,Head_orientation-NK-2023-07-17


In [26]:
(train.TrainingParamSet & "paramset_idx=1").fetch("params")

array([{'Task': 'Head_orientation', 'scorer': 'NK', 'date': 'Jul17', 'multianimalproject': 'False', 'identity': None, 'bodyparts': ['left_ear', 'right_ear', 'nose', 'head_middle', 'neck', 'body_middle', 'tail'], 'start': 0, 'stop': 1, 'numframes2pick': 100, 'skeleton': [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']], 'skeleton_color': 'black', 'pcutoff': 0.6, 'dotsize': 12, 'alphavalue': 0.7, 'colormap': 'rainbow', 'TrainingFraction': [0.95], 'iteration': 0, 'default_net_type': 'resnet_50', 'default_augmenter': 'default', 'snapshotindex': 3, 'batch_size': 8, 'cropping': False, 'x1': 0, 'x2': 640, 'y1': 277, 'y2': 624, 'corner2move2': [50, 50], 'move2corner': True, 'shuffle': '1', 'trainingsetindex': '0', 'maxiters': '5', 'scorer_legacy': 'False'}],
      dtype=object)

In [27]:
# (train.TrainingTask() & ("video_set_id = 1")).delete()
train.TrainingTask()

video_set_id,paramset_idx,training_id,model_prefix,project_path DLC's project_path in config relative to root
0,0,0,,from_top_tracking-DJ-2022-02-23/
1,1,1,,Head_orientation-NK-2023-07-17


In [28]:
train.ModelTraining.populate()

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'bodycenter', 'tailbase'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_from_top_trackingFeb23\\from_top_tracking_DJ95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': 'C:\\Users\\mini2p_1_aux\\.conda\\envs\\DEEPLABCUT_GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\mobilenet_v2_1.0_224.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref

Selecting single-animal trainer


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mini2p_1_aux\\DLC\\NEW\\from_top_tracking-DJ-2022-02-23/training-datasets\\iteration-0\\UnaugmentedDataSet_from_top_trackingFeb23\\from_top_tracking_DJ95shuffle1.mat'

In [ ]:
train.ModelTraining()

To start training from a previous instance, one would need to 
[edit the relevant config file](https://github.com/DeepLabCut/DeepLabCut/issues/70) and
adjust the `maxiters` paramset (if present) to a higher threshold (e.g., 10 for 5 more itterations).
Emperical work from the Mathis team suggests 200k iterations for any true use-case.

## Tracking Joints/Body Parts

The `model` schema uses a lookup table for managing Body Parts tracked across models.

In [ ]:
model.Model()

In [ ]:
model.BodyPart.heading

This table is equipped with two helper functions. First, we can identify all the new body parts from a given config file.

In [29]:
from adamacs.paths import get_dlc_root_data_dir
config_path = find_full_path(get_dlc_root_data_dir()[1], 
                             paramset_desc + '/config.yaml')
model.BodyPart.extract_new_body_parts(config_path)

Existing body parts: ['body_middle' 'head_middle' 'left_ear' 'neck' 'nose' 'right_ear' 'tail']
New body parts: []


array([], dtype='<U11')

Now, we can make a list of descriptions in the same order, and insert them into the table

In [30]:
model.BodyPart()

body_part,body_part_description
body_middle,
head_middle,
left_ear,
neck,
nose,
right_ear,
tail,


In [31]:
# Will be inserted with model insertion
# bp_desc=['Body Center', 'Head', 'Base of Tail']
# model.BodyPart.insert_from_config(config_path,bp_desc)

If we skip this step, body parts (without descriptions) will be added when we insert a model. We can [update](https://docs.datajoint.org/python/v0.13/manipulation/3-Cautious-Update.html) empty descriptions at any time.

## Declaring a Model

If training appears successful, the result can be inserted into the `Model` table for automatic evaluation.

In [32]:
paramset_desc='from_top_tracking-DJ-2022-02-23'

config_path = find_full_path(get_dlc_root_data_dir()[0], 
                             paramset_desc + '/config.yaml')

In [33]:
model.Model.insert_new_model(model_name='from_top_tracking-DJ',dlc_config=config_path,
                             shuffle=1,trainingsetindex=0,
                             model_description='From Top, trained 5 iterations',
                             paramset_idx=0)

IndexError: list index out of range

In [34]:
paramset_desc='Head_orientation-NK-2023-07-17'

config_path = find_full_path(get_dlc_root_data_dir()[1], 
                             paramset_desc + '/config.yaml')

In [35]:
model.Model.insert_new_model(model_name='Head_orientation-NK',dlc_config=config_path,
                             shuffle=1,trainingsetindex=0,
                             model_description='From Top, trained 5 iterations',
                             paramset_idx=1)

--- DLC Model specification to be inserted ---
	model_name: Head_orientation-NK
	model_description: From Top, trained 5 iterations
	scorer: DLC_resnet50_Head_orientationJul17shuffle1_90000
	task: Head_orientation
	date: Jul17
	iteration: 0
	snapshotindex: 3
	shuffle: 1
	trainingsetindex: 0
	project_path: Head_orientation-NK-2023-07-17
	paramset_idx: 1
	-- Template/Contents of config.yaml --
		Task: Head_orientation
		scorer: NK
		date: Jul17
		multianimalproject: False
		identity: None
		project_path: /datajoint-data/data/tobiasr/DeepLabCutModels/NK_DLC_tracking/Head_orientation-NK-2023-07-17
		video_sets: {'/datajoint-data/data/tobiasr/DeepLabCutModels/NK_DLC_tracking/Head_orientation-NK-2023-07-17/videos/scan9FHF1JT7_top_video_2023-04-06T09_31_19.mp4': {'crop': '0, 1000, 0, 1000'}}
		bodyparts: ['left_ear', 'right_ear', 'nose', 'head_middle', 'neck', 'body_middle', 'tail']
		start: 0
		stop: 1
		numframes2pick: 100
		skeleton: [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']]
		

In [36]:
model.Model()

model_name User-friendly model name,task Task in the config yaml,date Date in the config yaml,iteration Iteration/version of this model,"snapshotindex which snapshot for prediction (if -1, latest)",shuffle Shuffle (1) or not (0),trainingsetindex Index of training fraction list in config.yaml,scorer Scorer/network name - DLC's GetScorerName(),config_template Dictionary of the config for analyze_videos(),project_path DLC's project_path in config relative to root,model_prefix,model_description,paramset_idx
Head_orientation-NK,Head_orientation,Jul17,0,3,1,0,DLC_resnet50_Head_orientationJul17shuffle1_90000,=BLOB=,Head_orientation-NK-2023-07-17,,"From Top, trained 5 iterations",1


In [37]:
model.VideoRecording * model.VideoRecording.File()

session_id,recording_id,file_id,camera,"file_path filepath of video, relative to root data directory"
sess9FB2LN5C,scan9FB2LN5C,1,mini2p1_top,/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_top_video_2022-03-18T16_55_33.mp4
sess9FHELAYA,scan9FHELAYA,2,mini2p1_top,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4-copy.mp4
sess9FJ842C3,scan9FJ842C3,0,mini2p1_top,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-07-24_scan9FJ842C3_sess9FJ842C3/scan9FJ842C3_top_video_2023-07-24T16_38_06.mp4


In [38]:
model.Model().fetch("config_template")

array([{'Task': 'Head_orientation', 'scorer': 'NK', 'date': 'Jul17', 'multianimalproject': False, 'identity': None, 'project_path': '/datajoint-data/data/tobiasr/DeepLabCutModels/NK_DLC_tracking/Head_orientation-NK-2023-07-17', 'video_sets': {'/datajoint-data/data/tobiasr/DeepLabCutModels/NK_DLC_tracking/Head_orientation-NK-2023-07-17/videos/scan9FHF1JT7_top_video_2023-04-06T09_31_19.mp4': {'crop': '0, 1000, 0, 1000'}}, 'bodyparts': ['left_ear', 'right_ear', 'nose', 'head_middle', 'neck', 'body_middle', 'tail'], 'start': 0, 'stop': 1, 'numframes2pick': 100, 'skeleton': [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']], 'skeleton_color': 'black', 'pcutoff': 0.6, 'dotsize': 12, 'alphavalue': 0.7, 'colormap': 'rainbow', 'TrainingFraction': [0.95], 'iteration': 0, 'default_net_type': 'resnet_50', 'default_augmenter': 'default', 'snapshotindex': 3, 'batch_size': 8, 'cropping': False, 'x1': 0, 'x2': 640, 'y1': 277, 'y2': 624, 'corner2move2': [50, 50], 'move2corner': True}],
      dtype=

In [39]:
model.Model.BodyPart()

model_name User-friendly model name,body_part
Head_orientation-NK,body_middle
Head_orientation-NK,head_middle
Head_orientation-NK,left_ear
Head_orientation-NK,neck
Head_orientation-NK,nose
Head_orientation-NK,right_ear
Head_orientation-NK,tail


## Model Evaluation

Next, all inserted models can be evaluated with a similar `populate` method, which will
insert the relevant output from DLC's `evaluate_network` function.

In [40]:
model.ModelEvaluation.heading

model_name           : varchar(64)                  # User-friendly model name
---
train_iterations     : int                          # Training iterations
train_error=null     : float                        # Train error (px)
test_error=null      : float                        # Test error (px)
p_cutoff=null        : float                        # p-cutoff used
train_error_p=null   : float                        # Train error with p-cutoff
test_error_p=null    : float                        # Test error with p-cutoff

In [41]:
model.ModelEvaluation.populate()

If your project was initialized in a version of DeepLabCut other than the one you're currently using, model evaluation may report key errors. Specifically, your `config.yaml` may not specify `multianimalproject: false`.

In [ ]:
model.ModelEvaluation()

In [42]:
model.RecordingInfo.populate()
model.RecordingInfo()

session_id,recording_id,px_height height in pixels,px_width width in pixels,nframes number of frames,fps (Hz) frames per second,recording_datetime Datetime for the start of the recording,recording_duration video duration (s) from nframes / fps
sess9FB2LN5C,scan9FB2LN5C,500,500,18000,60,None,300.0
sess9FHELAYA,scan9FHELAYA,1000,1000,123,60,None,2.05
sess9FJ842C3,scan9FJ842C3,1000,1000,15851,60,None,264.183


## Pose Estimation

In [43]:
scansi = "scan9FHELAYA"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
path = (model.VideoRecording.File & scan_key).fetch("file_path")
path

array(['/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4-copy.mp4'],
      dtype=object)

For demonstration purposes, we'll make a shorter video that will process relatively quickly `ffmpeg`, a DLC dependency ([more info here](https://github.com/datajoint/workflow-deeplabcut/blob/main/notebooks/00-DataDownload_Optional.ipynb))

In [ ]:
from adamacs.paths import get_dlc_root_data_dir
vid_path =  find_full_path(get_dlc_root_data_dir(), path[0][1::])
print(vid_path)
cmd = (f'ffmpeg -n -hide_banner -loglevel error -ss 0 -t 2 -i {vid_path} '
       + f'-vcodec copy -acodec copy {vid_path}-copy.mp4')
import os; os.system(cmd)

Next, we need to specify if the `PoseEstimation` table should load results from an existing file or trigger the estimation command. Here, we can also specify parameters accepted by the `analyze_videos` function as a dictionary. `task_mode` determines if pose estimation results should be loaded or triggered (i.e., load vs. trigger).

In [44]:
key = (model.VideoRecording & {'recording_id': 'scan9FHELAYA'}).fetch1('KEY')
key.update({'model_name': 'Head_orientation-NK', 'task_mode': 'trigger'})
key



{'session_id': 'sess9FHELAYA',
 'recording_id': 'scan9FHELAYA',
 'model_name': 'Head_orientation-NK',
 'task_mode': 'trigger'}

The `PoseEstimationTask` table queues items for pose estimation. Additional parameters are passed to DLC's `analyze_videos` function.

In [45]:
model.PoseEstimationTask.insert_estimation_task(key, key["model_name"], analyze_videos_params={'save_as_csv':True})

In [46]:
model.RecordingInfo.populate()
model.RecordingInfo()

session_id,recording_id,px_height height in pixels,px_width width in pixels,nframes number of frames,fps (Hz) frames per second,recording_datetime Datetime for the start of the recording,recording_duration video duration (s) from nframes / fps
sess9FB2LN5C,scan9FB2LN5C,500,500,18000,60,None,300.0
sess9FHELAYA,scan9FHELAYA,1000,1000,123,60,None,2.05
sess9FJ842C3,scan9FJ842C3,1000,1000,15851,60,None,264.183


In [47]:
model.PoseEstimation.populate()

In [ ]:
model.PoseEstimation().

By default, DataJoint will store the results of pose estimation in a subdirectory
>  processed_dir / videos / device_<#>_recording_<#>_model_<name>

Pulling processed_dir from `get_dlc_processed_dir`, and device/recording information 
from the `VideoRecording` table. The model name is taken from the primary key of the
`Model` table, with spaced replaced by hyphens.
    
We can get this estimation directly as a pandas dataframe.

In [ ]:
model.PoseEstimation.BodyPartPosition()

In [ ]:
model.PoseEstimation.get_trajectory(key)

In [ ]:
df=model.PoseEstimation.get_trajectory(key)
df_xy = df.iloc[:,df.columns.get_level_values(2).isin(["x","y"])]['Head_orientation-NK']
df_xy.mean()


In [ ]:
df_xy.plot().legend(loc='right')

In [ ]:
df_flat = df_xy.copy()
df_flat.columns = df_flat.columns.map('_'.join)

In [ ]:
import matplotlib.pyplot as plt 
fig,ax=plt.subplots()
df_flat.plot(x='body_middle_x',y='body_middle_y',ax=ax)
df_flat.plot(x='head_middle_x',y='head_middle_y', ax=ax)
df_flat.plot(x='tail_x',y='tail_y', ax=ax)

In [49]:
destfolder = model.PoseEstimationTask.infer_output_dir(key)
destfolder

PosixPath('/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/device_mini2p1_top_recording_scan9FHELAYA_model_Head_orientation-NK')

In [50]:
from deeplabcut.utils.make_labeled_video import create_labeled_video

video_path = find_full_path( # Fetch the full video path
    get_dlc_root_data_dir(), ((model.VideoRecording.File & key).fetch1("file_path"))
)

config_paths = sorted( # Of configs in the project path, defer to the datajoint-saved
    list(
        find_full_path(
            get_dlc_root_data_dir(), ((model.Model & key).fetch1("project_path"))
        ).glob("*.y*ml")
    )
)

create_labeled_video( # Pass strings to label the video
    config=str(config_paths[-1]),
    videos=str(video_path),
    destfolder=str(destfolder),
)

# list(list(pathlib.Path((model.VideoRecording.File & key).fetch1("file_path")).parent.glob("device*"))[0].glob("*.y*ml"))

Starting to process video: /datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4-copy.mp4
Loading /datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4-copy.mp4 and data.


Duration of video [s]: 2.05, recorded with 60.0 fps!
Overall # of frames: 123 with cropped frame dimensions: 1000 1000
Generating frames and creating video.


100%|██████████| 123/123 [00:00<00:00, 136.35it/s]


[True]

In [ ]:
moviepath = str(list(pathlib.Path((model.VideoRecording.File & key).fetch1("file_path")).glob("*53.mp4"))[0])
moviepath


In [ ]:
(model.VideoRecording.File & key).fetch1("file_path")

In [ ]:
list(list(pathlib.Path((model.VideoRecording.File & key).fetch1("file_path")).parent.glob("device*"))[0].glob("*.y*ml"))